In [ ]:
'''
Вы играете в интересную стратегию. У вашего соперника остались всего одна казарма — здание, 
в котором постоянно появляются новые солдаты. Перед атакой у вас есть x солдат. 
За один раунд каждый солдат может убить одного из солдат противника или нанести 1 очко урона казарме 
(вычесть единицу здоровья у казармы). Изначально у вашего оппонента нет солдат. 
Тем не менее, его казарма имеет y единиц здоровья и производит p солдат за раунд.
Ход одного раунда:
 1 Каждый солдат из вашей армии либо убивает одного из солдат вашего противника, либо наносит 1 очко урона казарме. 
 Каждый солдат может выбрать своё действие. Когда казарма теряет все свои единицы здоровья, она разрушается.
 2 Ваш противник атакует. Он убьет k ваших солдат, где k — количество оставшихся у противника солдат.
 3 Если казармы еще не разрушены, ваш противник производит p новых солдат.

Ваша задача — разрушить казарму и убить всех солдат противника. Если это возможно, посчитайте минимальное количество раундов, 
которое вам нужно для этого. В противном случае выведите -1.
Формат ввода
На вход подаётся три целых числа x, y, p (1 ≤ x, y, p ≤ 5000) — количество ваших солдат на старте игры, 
количество очков здоровья казармы и количество производимых за раунд казармой солдат, соответственно. Каждое число расположено в новой строке.
Формат вывода
Если возможно убить всех вражеских солдат и разрушить казарму, выведите минимальное количество раундов, 
необходимых для этого. В противном случае выведите -1.
'''

In [8]:
from dataclasses import dataclass
import math

EPS = 1 # если на каких-то тестах будет падать, можно немного увеличить этот параметр

@dataclass(unsafe_hash=True)
class Game:
    x: int # число своих солдат
    y: int # здоровье вражеской казармы
    k: int # число вражеских солдат
    p: int # число производимых казармой солдат

    def winned(self):
        return self.y == 0 and self.k == 0

    def loosed(self):
        return self.x == 0


def enemy_step(game: Game):
    game.x = max(game.x - game.k, 0) # враг атакует
    if game.y > 0:
        game.k += game.p # враг генерит солдат


def game_step(i, games: set[Game]):
    i += 1
    # print(f'step = {i}, len(games) = {len(games)}')
    new_games = set()
    results = set()

    def check(new_game):
        if new_game.winned():
            results.add(i)
        elif not new_game.loosed():
            enemy_step(new_game)
            if not new_game.loosed():
                new_games.add(new_game)

    for game in games:
        #new_game = None
        if game.y > 0 and game.k > 0: # осталась и казарма и солдаты 
            # бьем врагов, остатки на казарму
            for m in range(min(game.x, game.k, EPS)):
                l = min(game.x, game.k) - m
                n = game.x - l
                new_game = Game(x=game.x, y=max(game.y - n, 0), k=max(game.k - l, 0), p=game.p)
                check(new_game)
            # бьем казарму, остатки на солдат
            for m in range(min(game.x, game.y, EPS)):
                n = min(game.x, game.y) - m
                l = game.x - n
                new_game = Game(x=game.x, y=max(game.y - n, 0), k=max(game.k - l, 0), p=game.p)
                check(new_game)
        elif game.y > 0: # осталась казарма, но нет солдат, значит всех своих солдат бросаем на поломку казармы
            new_game = Game(x=game.x, y=max((game.y - game.x), 0), k=game.k, p=game.p)
            check(new_game)
        elif game.k > 0: # остались солдаты, но нет казармы, значит всех своих солдат бросаем на удбийство вражеских солдат
            new_game = Game(x=game.x, y=game.y, k=max((game.k - game.x), 0), p=game.p)
            check(new_game)
        else:
            results.add(i)
    
    #print(f'i = {i}, new_games = {new_games}')

    # возвращаем только те исходы игры, которых не было в начале шага
    # так как во-первых они просто лишние (на следующем шаге получим то же самое, что есть сейчас)
    # а кроме того, позволить избавиться от зацикливания (то есть будем отбрасывать такие исходы игры, которые не меняются во время хода)
    return new_games - games, results, i

def game(x, y, p):
    if p == 1 and x > 10 and x < y:
        return math.ceil(y / (x - p))
    else:
        i = 0
        total_results = set()
        games = {Game(x=x, y=y, k=0, p=p)}
        while True:
            games, results, i = game_step(i, games)
            total_results |= results # объединение двух множеств
            if len(total_results) > 0:
                return min(total_results)
            if len(games) == 0:
                return -1
    
x = int(input())
y = int(input())
p = int(input())
print(game(x, y, p))

51
